In [1]:
## IMPORTS
import pandas as pd
import pandas_datareader.data as pdr
from pandas import Series, DataFrame
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import datetime
from datetime import date, timedelta
import yfinance as yf
yf.pdr_override()

def fullPrint(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(df)   

In [2]:
## TOOL FUNCTIONS
def fullPrint(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(df)   
def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iloc[index].Close/df.iloc[index-30].Close
def calculateSlope(df,index):
    if index < 31:
        return 0
    else:
        return df.iloc[index]['30WMA']/df.iloc[index-1]['30WMA']



In [3]:
#*Stage Checkers
def checkIfStage1(price,volumePerc, RS, slope, WMA,P4WH,P4WL):
    stageOneIndicator = 0
    falseReason = ""
    if volumePerc<0.7:
        stageOneIndicator+=1
    else:
        falseReason += "volume "
    if slope<=1.03 and slope>=0.995:
        stageOneIndicator+=1
    else:
        falseReason += "slope "
    if P4WH<P4WL*1.15:
        stageOneIndicator+=1
    else:
        falseReason += "range "
    if price <= WMA*1.1 and price>= WMA*0.9:
        stageOneIndicator+=1
    else:
        falseReason += "price "
    if stageOneIndicator>=3:
        return True
    return "False " + falseReason
def checkIfStage2(price,volumePerc, RS, slope, WMA,prevStage,prevClose):
    if prevStage != "Stage 2" and price > 150:
        return "Too Expansive"
    if volumePerc < 0.7:
        if prevStage != "Stage 2":
            #print(prevStage)
            return "Volume"
    if RS < 0.1 and prevStage != "Stage 2":
        return "RS"
    if RS < 0 and prevStage == "Stage 2":
        return "RS"
    if slope < 1.03 and prevStage != "Stage 2":
        return "Slope"
    if slope < 1.005 and prevStage == "Stage 2":
        return "Slope"
    if price < WMA*1.1 and prevStage != "Stage 2":
        return "Price"
    if price < WMA*0.95 and prevStage == "Stage 2":
        return "Price"
    return "Clear"
def checkStage(price,volumePerc, RS, slope, WMA,P4WH,P4WL,prevStage,prevClose):
    stage1Check = checkIfStage1(price,volumePerc, RS, slope, WMA,P4WH,P4WL)
    stage2Check = checkIfStage2(price,volumePerc, RS, slope, WMA,prevStage,prevClose)
    if stage2Check == "Clear":
        return "Stage 2"
    if stage1Check == True:
        return "Stage 1 " + stage2Check 
    return stage2Check    

In [4]:
def returnStageDf(dfSorted,spDfSorted):
    deltaX = 10.4
    weights = np.arange(1,31)
    sumWeights = np.sum(weights)
    dfSorted['30WMA'] = dfSorted['Close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
    dfSorted['30WMASlope'] = dfSorted.apply(lambda x: calculateSlope(dfSorted,dfSorted.index.get_loc(x.name)),axis=1)
    dfSorted['P4WH'] = dfSorted['Close'].rolling(window=4).max()
    dfSorted['P4WL'] = dfSorted['Close'].rolling(window=4).min()
    dfSorted['VolumePerc'] = dfSorted['Volume'].pct_change()
    dfSorted['Percent'] = dfSorted.apply(lambda x: product(dfSorted,dfSorted.index.get_loc(x.name)),axis=1)
    spDfSorted['Percent'] = dfSorted.apply(lambda x: product(spDfSorted,spDfSorted.index.get_loc(x.name)),axis=1)
    dfSorted['RS'] = dfSorted['Percent'] - spDfSorted['Percent']
    dfSorted = dfSorted.dropna()
    dfSorted['Stage'] = ""
    for index, element in dfSorted.iterrows():
        if dfSorted.index.get_loc(index) == 0:
            continue
        dfSorted.iloc[dfSorted.index.get_loc(index), dfSorted.columns.get_loc('Stage')] = checkStage(dfSorted.loc[index]['Close'],dfSorted.loc[index]['VolumePerc'],dfSorted.loc[index]['RS'],dfSorted.loc[index]['30WMASlope'],dfSorted.loc[index]['30WMA'],dfSorted.loc[index]['P4WH'],dfSorted.loc[index]['P4WL'],dfSorted.iloc[dfSorted.index.get_loc(index) - 1]['Stage'],dfSorted.iloc[dfSorted.index.get_loc(index) - 1]['Close'])
    return dfSorted[["Close","Stage"]]

def getStage(ticker):
    spDfSorted = pd.read_csv('stockData/S&P500.csv').set_index('Date')
    try:
        return returnStageDf(pd.read_csv('stockData/R3000/'+ticker+'.csv').set_index('Date'),spDfSorted)
    except:
        pass
    try:
        return returnStageDf(pd.read_csv('stockData/S&P500/'+ticker+'.csv').set_index('Date'),spDfSorted)
    except:
        pass
    return 0
    
    

In [5]:
transaction = pd.read_csv('stockData/S&P500.csv').set_index('Date').drop(['Close','Volume'],axis=1)
transaction['holding'] = np.empty((len(transaction), 0)).tolist()

In [6]:
R3000TXT = open("stockData/russell3000Symbols.txt", "r")
R3000List = R3000TXT.read()
dfR3000Tickers = R3000List.split(",")
R3000TXT.close()
dfSPYTickers = pd.read_csv('stockData/S&P500Symbols.csv')
dfR3000 = list(zip(dfR3000Tickers, ['not hold']*len(dfR3000Tickers)))
dfSPY = list(zip(dfSPYTickers['Symbol'], ['not hold']*len(dfR3000Tickers)))
dfReal = []
for i in dfR3000:
    if i not in dfReal:
       dfReal.append(i)
for i in dfSPY:
    if i not in dfReal:
       dfReal.append(i)

In [13]:
totalEarnings = 0
totalCost = 0
totalSPYEarings = 0
totalTime = 0
count = 0
totalPerc = 0
SPYPerc = 0
dfSPY = pd.read_csv('stockData/S&P500.csv').set_index('Date')
F500 = ['GM', 'F', 'XOM', 'T', 'GE', 'MO', 'DD', 'CVS', 'C', 'PG', 'PEP', 'AMCR', 'HPE', 'KR', 'MET', 'XLB', 'BA', 'TGT', 'ENNV', 'TKUMF', 'UPS', 'WINA', 'C', 'CNC', 'PYS', 'XRX', 'ETN', 'KODK', 'PCTY', 'MRO', 'COST', 'KO', 'WBREOX', 'JNJ', 'SIVB', 'AXP', 'SEAS', 'MMM', 'MRK', 'IP', 'CAT', 'VZ', 'LCID', 'NIO', 'LMT', 'BAC', 'SPMYY', 'ATVK', 'HD', 'MCK', 'COP', 'DAL', 'GT', 'BUD', 'BMY', 'JPM', 'ACI', 'SBAC', 'INTC', 'ADM', 'ROK', 'NUSI', 'CSCO', 'QREARX', 'IWM', 'WBA', 'TXN', 'WM', 'DIS', 'RTX', 'CSLT', 'MLM', 'TXT', 'ALGN', 'OXY', 'BAX', 'WBA', 'COUP', 'LEHLQ', 'ABT', 'TNA', 'TNA', 'MKFG.WT', 'ETHUSD', 'EMR', 'AOMOY', 'GIS', 'FDX', 'FBHS', 'EIX', 'MCD', 'SO', 'FL', 'PFE', 'UNP', 'WHR', 'PFG', 'CI', 'VIAC', 'CL', 'ANSS', 'KMB', 'AU', 'JNJ', 'LLY', 'OSRS', 'LNC', 'FMCKJ', 'JCI', 'DOW', 'NOC', 'WUMSY', 'CPB', 'MS', 'QCOM', 'SEE', 'DFS', 'PPG', 'XYL', 'EHC', 'ED', 'USB', 'AFL', 'LCID', 'LEVI', 'GS', 'HON', 'ETN', 'WFC', 'ICE', 'PEG', 'CAH', 'HAL', 'DDS', 'CMG', 'BTO', 'TXN', 'AEP', 'VOBIF', 'EVO', 'KEY', 'TWMTX', 'DOTUSD', 'CLR', 'ARMK', 'FB', 'TSN', 'MRZLF', 'FSRXU', 'W', 'WFC', 'RC.PRE', 'TCPIX', 'RHCO', 'RRD', 'GPC', 'GLW', 'CMI', 'R', 'PNC', 'ARW', 'MSFT', 'NSC', 'AKTX', 'LWLG', 'BKR', 'DOMR', 'DUK', 'MAS', 'CCK', 'RAD', 'EMN', 'ABC', 'MAN', 'FIBK', 'EXC', 'LRLCY', 'CNP', 'JWN', 'BRK.B', 'DLLFF', 'GESI', 'CP', 'PBI', 'NKE', 'SCL', 'UNH', 'GE', 'COF', 'COUP', 'GD', 'HUM', 'JLL', 'COF', 'UNM', 'CENX', 'CMS', 'CGNSF', 'PMCB', 'OVV', 'SFE', 'FUTU', 'RJF', 'TSM', 'DTP', 'RISR', 'PPWLM', 'STX', 'APD', 'ITW', 'THC', 'MARA', 'MPW', 'KMX', 'OC', 'HZNP', 'RIORX', 'DGII', 'JAZZ', 'SYUS', 'MET', 'LEA', 'FTMRQ', 'SHW', 'DOV', 'HRL', 'MPXOF', 'LPX', 'GWW', 'XOM', 'SPGI', 'BBY', 'NUE', 'UHS', 'ALLT', 'MODV', 'SHIP', 'FREE', 'AVY', 'BC', 'PFG', 'NURE', 'HTGC', 'AAPL', 'MGAWY', 'ELAMF', 'XM', 'KO', 'PARXF', 'PEBN', 'CAT', 'WMB', 'HAS', 'PSTH', 'WTM', 'ALGN', 'PKI', 'USO', 'DOTUSD', 'LUV', 'PH', 'TRV', 'PEI', 'BDX', 'CMA', 'F', 'BKR', 'XEL', 'RCII', 'ARVIX', 'DTRC', 'PGR', 'JNJ', 'TDY', 'USB', 'FE', 'KELYA', 'NYT', 'ACER', 'BLDP', 'HCYIX', 'SON', 'AES', 'YSACU', 'WING', 'JD', 'A', 'MCHP', 'FIJEX']
test = ['A','AAPL','GOOG','AA']
for symbol in dfReal:
    try:
        df = getStage(symbol[0])
        shares = 0
        sharesSPY = 0
        for index, element in df.iterrows():
            if shares == 0 and element.Stage == "Stage 2":
                shares = 1000/element.Close
                sharesSPY = 1000/dfSPY.loc[index].Close
                totalTime +=1
                totalCost +=1000
                count+=1
                transaction.loc[index]['holding'].append((symbol[0],element.Close,0))
                print("buy "+index)
                continue
            if shares != 0:
                if element.Stage == "Stage 2":
                    totalTime +=1
                    transaction.loc[index]['holding'].append((symbol[0],element.Close,0))
                else:
                    totalEarnings += shares*element.Close
                    totalSPYEarings += sharesSPY*dfSPY.loc[index].Close
                    totalPerc += shares*element.Close/1000
                    SPYPerc += sharesSPY*dfSPY.loc[index].Close/1000
                    print(symbol[0]+" "+str(shares*element.Close))
                    print("SPY "+ str(sharesSPY*dfSPY.loc[index].Close))
                    print("sell "+index)
                    shares = 0
                    transaction.loc[index]['holding'].append((symbol[0],element.Close,-1))
        if shares !=0:
            totalEarnings += df.iloc[-1].Close
            totalSPYEarings += sharesSPY*dfSPY.iloc[-1].Close
            totalPerc += shares*element.Close/1000
            SPYPerc += sharesSPY*dfSPY.loc[index].Close/1000
            transaction.loc[index]['holding'].append((symbol[0],element.Close,-1))
    except:
        print(symbol[0]+" Stock Not Found")
print(totalPerc/count)
print(SPYPerc/count)
print(totalTime/count)
print((totalTime/500)/(10*52))
print(count)

buy 2001-04-20
FLWS 1381.578964704952
SPY 878.9683311315148
sell 2001-09-14
buy 2001-06-29
SRCE 741.040228602155
SPY 992.8944027694182
sell 2001-07-13
FUBC Stock Not Found
buy 2000-01-28
DDD 862.5000822544176
SPY 997.3532705744375
sell 2000-04-14
buy 2000-06-09
DDD 1033.8983741240907
SPY 943.1827379108117
sell 2000-10-13
buy 2004-01-09
DDD 766.519834430374
SPY 1020.5729381170993
sell 2004-02-27
buy 2004-11-12
DDD 1055.587101280385
SPY 964.912092616621
sell 2005-04-15
buy 2009-09-18
DDD 895.1384419013123
SPY 1018.1409753786602
sell 2009-10-16
buy 2010-10-29
DDD 1429.9535393757167
SPY 1130.5799304383063
sell 2011-05-13
buy 2013-01-04
DDD 859.6193633494391
SPY 1057.7646216425399
sell 2013-03-08
buy 2014-01-10
DDD 703.4410226377753
SPY 975.3849792896497
sell 2014-02-07
buy 2016-04-15
DDD 767.346946946457
SPY 988.6625904792746
sell 2016-05-06
buy 2018-08-10
DDD 650.4024572945294
SPY 961.0981020007126
sell 2018-11-02
buy 2020-12-25
DDD 2069.264002064891
SPY 1114.9697113622906
sell 2021-04-09

In [1]:
# fullPrint(transaction)
transaction.to_csv('Testdata/Weekly/'+date.today().strftime('%Y-%m-%d')+'full.csv')
transactionData = transaction.copy()
test = pd.DataFrame()
test['holding'] = ([('F',50,0)],[('F',40,0)],[('F',30,-1)],[],[('M',25,0)],[('M',30,-1)])
test['total'] = (0,0,0,0,0,0)
testData = test.copy()
#fullPrint(transactionData)

NameError: name 'transaction' is not defined

In [9]:
transaction = transactionData.copy()
total = 100
first = True
prevIndex = ''
transaction['total'] = 0
transaction = transaction.astype({'total': 'int64'})
for index, element in transaction.iterrows():
    try:
        if len(element.holding) == 0:
            transaction['total'][index] = total
            continue
        if first: 
            first = not first
            transaction['total'][index] = total
        else:
            prevData = transaction.loc[prevIndex].holding
            if len(prevData):
                total = 0
            for i in range(len(prevData)):
                for j in range(len(element.holding)):
                    if element.holding[j][0] == prevData[i][0]:
                        close = element.holding[j][1]
                total += close*prevData[i][2]
            transaction['total'][index] = total
            for k in element.holding:
                if k[2] == -1:
                    element.holding.remove(k)
        count = len(element.holding)
        for i in range(len(element.holding)):
            element.holding[i] = (element.holding[i][0],element.holding[i][1], total/count/element.holding[i][1])
        prevIndex = index
    except:
        print(index)
        break
        

In [10]:
fullPrint(transaction['total'])

Date
1995-01-06      100
1995-01-13      100
1995-01-20      100
1995-01-27      100
1995-02-03      100
1995-02-10      100
1995-02-17      100
1995-02-24      100
1995-03-03      100
1995-03-10      100
1995-03-17      100
1995-03-24      100
1995-03-31      100
1995-04-07      100
1995-04-14      100
1995-04-21      100
1995-04-28      100
1995-05-05      100
1995-05-12      100
1995-05-19      100
1995-05-26      100
1995-06-02      100
1995-06-09      100
1995-06-16      100
1995-06-23      100
1995-06-30      100
1995-07-07      100
1995-07-14      100
1995-07-21      100
1995-07-28      100
1995-08-04      100
1995-08-11      100
1995-08-18      100
1995-08-25      100
1995-09-01      100
1995-09-08      100
1995-09-15      106
1995-09-22      106
1995-09-29       90
1995-10-06       87
1995-10-13       87
1995-10-20       81
1995-10-27       81
1995-11-03       81
1995-11-10       81
1995-11-17       81
1995-11-24       81
1995-12-01       81
1995-12-08       81
1995-12-15     

In [5]:
import pandas as pd
test = pd.DataFrame()
test['holding'] = ([('F',50,0),('APPL',60,0)],[('F',40,0),('APPL',70,-1)],[('F',30,-1)],[],[('M',25,0)],[('M',30,-1)])
test['total'] = (0,0,0,0,0,0)
testData = test.copy()
test = testData.copy()
total = 100
first = True
prevIndex = ''
test['total'] = 0
test = test.astype({'total': 'int64'})
print(test.dtypes)
for index, element in test.iterrows():
    if len(element.holding) == 0:
        test['total'][index] = total
        continue
    if first: 
        first = not first
        test['total'][index] = total
    else:
        prevData = test.loc[prevIndex].holding
        if len(prevData):
            total = 0
        for i in range(len(prevData)):
            for j in range(len(element.holding)):
                if element.holding[j][0] == prevData[i][0]:
                    close = element.holding[j][1]
            total += close*prevData[i][2]
        test['total'][index] = total
        for k in element.holding:
            if k[2] == -1:
                element.holding.remove(k)
    count = len(element.holding)
    for i in range(len(element.holding)):
        element.holding[i] = (element.holding[i][0],element.holding[i][1], total/count/element.holding[i][1])
    prevIndex = index
        

holding    object
total       int64
dtype: object


/var/folders/n5/0wg686hs5jn0glfrk05fk_6r0000gn/T/ipykernel_1119/2514595718.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['total'][index] = total
/var/folders/n5/0wg686hs5jn0glfrk05fk_6r0000gn/T/ipykernel_1119/2514595718.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['total'][index] = total
/var/folders/n5/0wg686hs5jn0glfrk05fk_6r0000gn/T/ipykernel_1119/2514595718.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [6]:
print(test)

                                          holding  total
0  [(F, 50, 1.0), (APPL, 60, 0.8333333333333334)]    100
1                   [(F, 40, 2.4583333333333335)]     98
2                                              []     73
3                                              []     73
4                                 [(M, 25, 2.95)]     73
5                                              []     88
